Imports

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Nifti Import

**From Directory**
___


### 1. Cortical Thickness (CTh)
- **path**: Absolute path to the base directory containing CTh files.
- **glob_name_pattern**: File pattern to search for CTh files.

---

**Instructions**: Please fill out the `path` and `glob_name_pattern` fields for each segment in the `segments_dict`. This will ensure that the extraction process can locate and identify the appropriate neuroimaging files for further analysis.
---

# Import Segmented Patients for Atrophy Detection

In [ ]:
base_directory = r'/Volumes/OneTouch/datasets/adni/neuroimaging/true_ad_randomized/vbm/raws'
ct_pattern = '*/mri/*mwp1*resampled*'

In [1]:
from calvin_utils.file_utils.import_matrices import import_matrices_from_folder #<----- CALVIN IMPORT

def import_ct_dataframes_from_folders(base_directory, ct_glob_name_pattern):
    """
    Imports dataframes from specified directories and glob name patterns.
    
    Parameters:
    - base_directory (str): The base directory where the data resides.
    - grey_matter_glob_name_pattern (str): Glob pattern for grey matter data.
    - white_matter_glob_name_pattern (str): Glob pattern for white matter data.
    - csf_glob_name_pattern (str): Glob pattern for cerebrospinal fluid data.
    
    Returns:
    - dict: A dictionary containing dataframes for grey matter, white matter, and cerebrospinal fluid.
    """
    

    segments_dict = {
        'ct': {'path': base_directory, 'glob_name_pattern': ct_glob_name_pattern},
    }

    dataframes_dict = {}

    for k, v in segments_dict.items():
        dataframes_dict[k] = import_matrices_from_folder(connectivity_path=v['path'], file_pattern=v['glob_name_pattern'])
        print(f'Imported data {k} data with {dataframes_dict[k].shape[0]} voxels and {dataframes_dict[k].shape[1]} patients')
        print(f'These are the filenames per subject {dataframes_dict[k].columns}')
        print('--------------------------------')

    return dataframes_dict


/usr/local/Caskroom/mambaforge/base/envs/nimlab_py310/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
dataframes_dict = import_ct_dataframes_from_folders(base_directory, ct_pattern)

**Extract Subject ID From File Names**
- Using the example filenames that have been printed above, please define a general string:
1) Preceding the subject ID. If nothing preceding subject identifier, enter "".
- Do NOT include mwp[1/2/3] in this. 
2) Proceeding the subject ID. If nothing proceeding subject identifier, enter "".

In [ ]:
preceding_id = ''
proceeding_id = '_resam'

In [ ]:
from calvin_utils.vbm_utils.preprocessing import rename_dataframe_subjects
dataframes_dict = rename_dataframe_subjects(dataframes_dict, preceding_id, proceeding_id)

# Import Control Segments

In [ ]:
base_directory_control = '/Volumes/Expansion/datasets/adni/neuroimaging/true_control/freesurfer_reconall_sbm'
control_cortical_thickness_glob_name_pattern = '*/vol/*_vol_MNI152_s6.nii.gz'

In [ ]:
from calvin_utils.vbm_utils.preprocessing import import_control_dataframes
control_dataframes_dict = import_ct_dataframes_from_folders(base_directory_control, control_cortical_thickness_glob_name_pattern)

**Extract Subject ID From File Names**
- Using the example filenames that have been printed above, please define a general string:
1) Preceding the subject ID. If nothing preceding subject identifier, enter "".
- **Do NOT include mwp[1/2/3] in this.**
2) Proceeding the subject ID. If nothing proceeding subject identifier, enter "".


- The example filenames were all provided above

In [ ]:
preceding_id = 's'
proceeding_id = '_re'

In [ ]:
from calvin_utils.vbm_utils.preprocessing import rename_dataframe_subjects
control_dataframes_dict = rename_dataframe_subjects(control_dataframes_dict, preceding_id, proceeding_id)

# Import Covariates
Expects a CSV as below: 
```
+---------+----------------------------+--------------+--------------+--------------+
|Covariate| Subject 1                  | Subject 2    | Subject . . .| Subject N    |
+---------+----------------------------+--------------+--------------+--------------+
| Male    | 0                          | 1            | 1            | 1            |
| Female  | 1                          | 0            | 0            | 0            |
| Age     | 65                         | 72           | 87           | 90           |
+---------+----------------------------+--------------+--------------+--------------+
```
**1 is True, 0 is False, Age is represented in years.**

In [ ]:
control_covariates_csv_path = '/Users/cu135/Dropbox (Partners HealthCare)/studies/atrophy_seeds_2023/metadata/true_ad_patient_data/patient_covariates.csv'
patient_covariates_csv_path = '/Users/cu135/Dropbox (Partners HealthCare)/studies/atrophy_seeds_2023/metadata/paths_and_covariates/control_group_covariates.csv'

In [ ]:
from calvin_utils.vbm_utils.w_scoring import import_covariates
patient_covariates_df, control_covariates_df = import_covariates(control_covariates_csv_path=control_covariates_csv_path, patient_covariates_csv_path=patient_covariates_csv_path)
control_covariates_df

In [ ]:
patient_covariates_df

Improve the Naming of the Covariate Subjects
- Pending construction. Should have named it appropraitely when you built your CSV. 
- Will code PRN. 

# Generate W-Scored Atrophy Maps for Each Segment

In [ ]:
from calvin_utils.vbm_utils.w_scoring import CalvinWMap
wmapper = CalvinWMap(dataframes_dict=dataframes_dict, control_dataframes_dict=control_dataframes_dict, control_covariates_df=control_covariates_df, patient_covariates_df=patient_covariates_df)
atrophy_dataframes_dict, significant_atrophy_dataframes_dict = wmapper.run()

**Generate Composite Maps**

In [ ]:
from calvin_utils.vbm_utils.processing import generate_composite_maps
unthresholded_atrophy_dataframes_dict = generate_composite_maps(atrophy_dataframes_dict, thresholded=False)
thresholded_atrophy_dataframes_dict = generate_composite_maps(significant_atrophy_dataframes_dict, thresholded=True)

# Save the W-Scored Maps

Unthresholded Maps

In [ ]:
base_directory='/Volumes/OneTouch/datasets/adni/neuroimaging/true_ad_randomized/vbm/w_atrophy_seeds'

In [ ]:
from calvin_utils.vbm_utils.processing import save_nifti_to_bids
save_nifti_to_bids(unthresholded_atrophy_dataframes_dict, bids_base_dir=base_directory, analysis='tissue_segment_w_scores', dry_run=False);

Thresholded Maps - The 'Real' Atrophy


In [ ]:
from calvin_utils.vbm_utils.processing import save_nifti_to_bids
save_nifti_to_bids(thresholded_atrophy_dataframes_dict, bids_base_dir=base_directory, analysis='thresholded_tissue_segment_w_scores', dry_run=False);

All Done. Enjoy your atrophy seeds.

--Calvin